In [5]:
import requests
import pandas as pd
from tqdm import tqdm_notebook
from time import sleep
import os
import json
from IPython.display import clear_output

In [7]:
endpoint = 'https://free-nba.p.rapidapi.com/'
headers = {
	"X-RapidAPI-Key": "563b99e32emsh860d6965169a64dp1da03bjsnf21eaf721a45",
	"X-RapidAPI-Host": "free-nba.p.rapidapi.com"
}

players_endpoint = os.path.join(endpoint, 'games')

querystring = {'page': 0, 'per_page': 50}

responses = []
max_pages = float('inf')
i = 0
num_timeouts = 0
success = 0

while i < max_pages:
  querystring['page'] = i
  response = requests.get(players_endpoint, headers=headers, params=querystring)
  while response.status_code == 429:
    num_timeouts += 1
    if(num_timeouts % 20 == 0):   
        print(f'Uh oh! We got put in time out >.< {num_timeouts}th time')
    sleep(10)
    response = requests.get(players_endpoint, headers=headers, params=querystring)
  response = json.loads(response.text)

  if max_pages == float('inf'):
    max_pages = response['meta']['total_pages']
  i += 1
  responses.extend(response['data'])
  success += 1
  print(f'collected {success} pages')
  clear_output(wait = True)

print('successfully collected data')

successfully collected data


In [8]:
df = pd.DataFrame(responses)

In [9]:
home_team_series = pd.DataFrame(df['home_team'].to_list())

In [10]:
home_team_series.columns = 'home_team_' + home_team_series.columns

In [11]:
df1 = pd.concat([df, home_team_series], axis=1)

In [12]:
df1 = df1.drop('home_team', axis=1)

In [13]:
columns = df1.columns.tolist()
new_columns = columns[:2] + columns[10:] + columns[2:10]
df2 = df1[new_columns]

In [30]:
visitor_team_series = pd.DataFrame(df['visitor_team'].to_list())
visitor_team_series.columns = 'visitor_team_' + visitor_team_series.columns
df3 = pd.concat([df2, visitor_team_series], axis = 1)
cleaned_df = df3.drop('visitor_team', axis = 1)
cleaned_df['date'] = pd.to_datetime(cleaned_df['date'])

In [31]:
cleaned_df

,id,date,home_team_id,home_team_abbreviation,home_team_city,home_team_conference,home_team_division,home_team_full_name,home_team_name,home_team_score,...,status,time,visitor_team_score,visitor_team_id,visitor_team_abbreviation,visitor_team_city,visitor_team_conference,visitor_team_division,visitor_team_full_name,visitor_team_name
0,47179,2019-01-30 00:00:00+00:00,2,BOS,Boston,East,Atlantic,Boston Celtics,Celtics,126,...,Final,,94,4,CHA,Charlotte,East,Southeast,Charlotte Hornets,Hornets
1,48751,2019-02-09 00:00:00+00:00,2,BOS,Boston,East,Atlantic,Boston Celtics,Celtics,112,...,Final,,123,13,LAC,LA,West,Pacific,LA Clippers,Clippers
2,48739,2019-02-08 00:00:00+00:00,23,PHI,Philadelphia,East,Atlantic,Philadelphia 76ers,76ers,117,...,Final,,110,8,DEN,Denver,West,Northwest,Denver Nuggets,Nuggets
3,48740,2019-02-08 00:00:00+00:00,30,WAS,Washington,East,Southeast,Washington Wizards,Wizards,119,...,Final,,106,6,CLE,Cleveland,East,Central,Cleveland Cavaliers,Cavaliers
4,48746,2019-02-08 00:00:00+00:00,26,SAC,Sacramento,West,Pacific,Sacramento Kings,Kings,102,...,Final,,96,16,MIA,Miami,East,Southeast,Miami Heat,Heat
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
70445,1038668,2024-03-30 00:00:00+00:00,1,ATL,Atlanta,East,Southeast,Atlanta Hawks,Hawks,0,...,2024-03-30T23:30:00Z,None,0,17,MIL,Milwaukee,East,Central,Milwaukee Bucks,Bucks
70446,1038669,2024-03-31 00:00:00+00:00,8,DEN,Denver,West,Northwest,Denver Nuggets,Nuggets,0,...,2024-03-31T19:30:00Z,None,0,6,CLE,Cleveland,East,Central,Cleveland Cavaliers,Cavaliers
70447,1038670,2024-03-31 00:00:00+00:00,3,BKN,Brooklyn,East,Atlantic,Brooklyn Nets,Nets,0,...,2024-03-31T22:00:00Z,None,0,14,LAL,Los Angeles,West,Pacific,Los Angeles Lakers,Lakers
70448,1038671,2024-03-31 00:00:00+00:00,4,CHA,Charlotte,East,Southeast,Charlotte Hornets,Hornets,0,...,2024-03-31T22:00:00Z,None,0,13,LAC,LA,West,Pacific,LA Clippers,Clippers


In [32]:
cleaned_df.to_csv('games.csv')

In [33]:
cleaned_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 70450 entries, 0 to 70449
Data columns (total 23 columns):
 #   Column                     Non-Null Count  Dtype              
---  ------                     --------------  -----              
 0   id                         70450 non-null  int64              
 1   date                       70450 non-null  datetime64[ns, UTC]
 2   home_team_id               70450 non-null  int64              
 3   home_team_abbreviation     70450 non-null  object             
 4   home_team_city             70450 non-null  object             
 5   home_team_conference       70450 non-null  object             
 6   home_team_division         70450 non-null  object             
 7   home_team_full_name        70450 non-null  object             
 8   home_team_name             70450 non-null  object             
 9   home_team_score            70450 non-null  int64              
 10  period                     70450 non-null  int64              
 11  po

In [34]:
def normalize_column(df, column_name):
    # Check if the column exists in the DataFrame
    if column_name not in df.columns:
        print(f"Column '{column_name}' not found in the DataFrame.")
        return df  # Return the original DataFrame

    min_val = df[column_name].min()
    max_val = df[column_name].max()

    if min_val == max_val:
        # Handle the case where min and max are the same to avoid division by zero
        df[column_name] = 0.0
    else:
        df[column_name] = (df[column_name] - min_val) / (max_val - min_val)

    return df

In [35]:
normalized_df = normalize_column(cleaned_df, 'home_team_score')
normalized_df = normalize_column(normalized_df, 'visitor_team_score')

In [36]:
normalized_df

,id,date,home_team_id,home_team_abbreviation,home_team_city,home_team_conference,home_team_division,home_team_full_name,home_team_name,home_team_score,...,status,time,visitor_team_score,visitor_team_id,visitor_team_abbreviation,visitor_team_city,visitor_team_conference,visitor_team_division,visitor_team_full_name,visitor_team_name
0,47179,2019-01-30 00:00:00+00:00,2,BOS,Boston,East,Atlantic,Boston Celtics,Celtics,0.684783,...,Final,,0.505376,4,CHA,Charlotte,East,Southeast,Charlotte Hornets,Hornets
1,48751,2019-02-09 00:00:00+00:00,2,BOS,Boston,East,Atlantic,Boston Celtics,Celtics,0.608696,...,Final,,0.661290,13,LAC,LA,West,Pacific,LA Clippers,Clippers
2,48739,2019-02-08 00:00:00+00:00,23,PHI,Philadelphia,East,Atlantic,Philadelphia 76ers,76ers,0.635870,...,Final,,0.591398,8,DEN,Denver,West,Northwest,Denver Nuggets,Nuggets
3,48740,2019-02-08 00:00:00+00:00,30,WAS,Washington,East,Southeast,Washington Wizards,Wizards,0.646739,...,Final,,0.569892,6,CLE,Cleveland,East,Central,Cleveland Cavaliers,Cavaliers
4,48746,2019-02-08 00:00:00+00:00,26,SAC,Sacramento,West,Pacific,Sacramento Kings,Kings,0.554348,...,Final,,0.516129,16,MIA,Miami,East,Southeast,Miami Heat,Heat
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
70445,1038668,2024-03-30 00:00:00+00:00,1,ATL,Atlanta,East,Southeast,Atlanta Hawks,Hawks,0.000000,...,2024-03-30T23:30:00Z,None,0.000000,17,MIL,Milwaukee,East,Central,Milwaukee Bucks,Bucks
70446,1038669,2024-03-31 00:00:00+00:00,8,DEN,Denver,West,Northwest,Denver Nuggets,Nuggets,0.000000,...,2024-03-31T19:30:00Z,None,0.000000,6,CLE,Cleveland,East,Central,Cleveland Cavaliers,Cavaliers
70447,1038670,2024-03-31 00:00:00+00:00,3,BKN,Brooklyn,East,Atlantic,Brooklyn Nets,Nets,0.000000,...,2024-03-31T22:00:00Z,None,0.000000,14,LAL,Los Angeles,West,Pacific,Los Angeles Lakers,Lakers
70448,1038671,2024-03-31 00:00:00+00:00,4,CHA,Charlotte,East,Southeast,Charlotte Hornets,Hornets,0.000000,...,2024-03-31T22:00:00Z,None,0.000000,13,LAC,LA,West,Pacific,LA Clippers,Clippers
